In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('sentiment_dataset.csv')

In [3]:
df

,text,label,src
0,"Пальто красивое, но пришло с дырой в молнии. П...",0,rureviews
1,"Очень долго шел заказ,ждала к новому году,приш...",0,rureviews
2,"Могу сказать одно, брюки нормальные, НО они бы...",0,rureviews
3,"Доставка быстрая, меньше месяца. Заказывали ра...",0,rureviews
4,Мне не очень понравилось это платье. Размер ...,0,rureviews
...,...,...,...
290453,Как всегда вкусный и свежий.,1,perekrestok
290454,Обалденный!,1,perekrestok
290455,"Достоинства:\nВкусно, лучшая паста\n\nНедостат...",1,perekrestok
290456,"Достоинства:\nУвлажняет очень хорошо, и кожа к...",1,perekrestok


In [4]:
df['label'].unique()

array([0, 1, 2], dtype=int64)

In [5]:
TARGET_SIZE = 10000
LABEL_COL = "label"

# сколько строк брать из каждого класса
n_classes = df[LABEL_COL].nunique()
samples_per_class = TARGET_SIZE // n_classes

# стратифицированная выборка
df_small = (
    df
    .groupby(LABEL_COL, group_keys=False)
    .apply(lambda x: x.sample(n=samples_per_class, random_state=42))
)

In [6]:
df_small

,text,label,src
271680,"Рассыпается на мелкие кусочки , когда в охлажд...",0,perekrestok
127957,Были в данном заведении в середине апреля. В ц...,0,geo
6843,"Заказала 11 сентября, пришли уже 25, цвет отли...",0,rureviews
194738,хочешь себе такую?))),0,rusentiment
146196,В целом хорошая точка пятерочки. Есть кассы са...,0,geo
...,...,...,...
241032,Ужасный водянистый вкус,2,perekrestok
118000,"Забавное аниме в 2-х минутах, на такие и ненар...",2,anime
80335,Очень большой размер,2,rureviews
219451,Подруга порекомендовала данный фильм к просмо...,2,kinopoisk


# Очистка

In [7]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^а-яё ]", " ", text)
    return text

In [8]:
df_small['clean_text'] = df_small['text'].apply(clean_text)

# Токенизация

In [9]:
from nltk.tokenize import word_tokenize

def tokinazer(text):
    tokens = word_tokenize(text, language="russian")
    return tokens

In [10]:
df_small['text_tokens'] = df_small['clean_text'].apply(lambda x: x.split())

In [105]:
df_small['text_tokens'] = df_small['clean_text'].apply(tokinazer)

# Лемматизация

In [106]:
from tqdm import tqdm
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

def lemmatize(tokens):
    return [
        morph.parse(word)[0].normal_form
        for word in tqdm(tokens, desc="Lemmatizing")
    ]

In [107]:
df_small

,text,label,src,clean_text,text_tokens
271680,"Рассыпается на мелкие кусочки , когда в охлажд...",0,perekrestok,рассыпается на мелкие кусочки когда в охлажд...,"[рассыпается, на, мелкие, кусочки, когда, в, о..."
127957,Были в данном заведении в середине апреля. В ц...,0,geo,были в данном заведении в середине апреля в ц...,"[были, в, данном, заведении, в, середине, апре..."
6843,"Заказала 11 сентября, пришли уже 25, цвет отли...",0,rureviews,заказала сентября пришли уже цвет отли...,"[заказала, сентября, пришли, уже, цвет, отлича..."
194738,хочешь себе такую?))),0,rusentiment,хочешь себе такую,"[хочешь, себе, такую]"
146196,В целом хорошая точка пятерочки. Есть кассы са...,0,geo,в целом хорошая точка пятерочки есть кассы са...,"[в, целом, хорошая, точка, пятерочки, есть, ка..."
...,...,...,...,...,...
241032,Ужасный водянистый вкус,2,perekrestok,ужасный водянистый вкус,"[ужасный, водянистый, вкус]"
118000,"Забавное аниме в 2-х минутах, на такие и ненар...",2,anime,забавное аниме в х минутах на такие и ненар...,"[забавное, аниме, в, х, минутах, на, такие, и,..."
80335,Очень большой размер,2,rureviews,очень большой размер,"[очень, большой, размер]"
219451,Подруга порекомендовала данный фильм к просмо...,2,kinopoisk,подруга порекомендовала данный фильм к просмо...,"[подруга, порекомендовала, данный, фильм, к, п..."


In [109]:
df_small['text_lemma'] = df_small['text_tokens'].apply(lemmatize)

Lemmatizing: 100%|█████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 3128.03it/s]
Lemmatizing: 0it [00:00, ?it/s]
Lemmatizing: 100%|███████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 3545.68it/s]
Lemmatizing: 0it [00:00, ?it/s]
Lemmatizing: 100%|███████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<?, ?it/s]
Lemmatizing: 0it [00:00, ?it/s]
Lemmatizing: 100%|███████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 3588.45it/s]
Lemmatizing: 0it [00:00, ?it/s]
Lemmatizing: 100%|███████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 4784.03it/s]
Lemmatizing: 0it [00:00, ?it/s]
Lemmatizing: 100%|███████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 2281.59it/s]
Lemmatizing: 0it [00:00, ?it/s]
Lemmatizing: 100%|███████████████████████████████████████████████████████████████████| 3

In [110]:
df_small

# Векторизация

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=3000)
df_small['text_string'] = df_small['text_lemma'].apply(lambda tokens: ' '.join(tokens))

# Теперь используйте новый столбец
X = vectorizer.fit_transform(df_small['text_string'])

In [62]:
df_small

,text,label,src,clean_text,text_tokens,text_lemma,text_string
271680,"Рассыпается на мелкие кусочки , когда в охлажд...",0,perekrestok,рассыпается на мелкие кусочки когда в охлажд...,"[рассыпается, на, мелкие, кусочки, когда, в, о...","[рассыпаться, на, мелкий, кусочек, когда, в, о...",рассыпаться на мелкий кусочек когда в охладить...
127957,Были в данном заведении в середине апреля. В ц...,0,geo,были в данном заведении в середине апреля в ц...,"[были, в, данном, заведении, в, середине, апре...","[быть, в, данный, заведение, в, середина, апре...",быть в данный заведение в середина апрель в це...
6843,"Заказала 11 сентября, пришли уже 25, цвет отли...",0,rureviews,заказала сентября пришли уже цвет отли...,"[заказала, сентября, пришли, уже, цвет, отлича...","[заказать, сентябрь, прислать, уже, цвет, отли...",заказать сентябрь прислать уже цвет отличаться...
194738,хочешь себе такую?))),0,rusentiment,хочешь себе такую,"[хочешь, себе, такую]","[хотеть, себя, такой]",хотеть себя такой
146196,В целом хорошая точка пятерочки. Есть кассы са...,0,geo,в целом хорошая точка пятерочки есть кассы са...,"[в, целом, хорошая, точка, пятерочки, есть, ка...","[в, целое, хороший, точка, пятёрочка, есть, ка...",в целое хороший точка пятёрочка есть касса сам...
...,...,...,...,...,...,...,...
113301,"лол, смотрю сейчас первую историю. история про...",2,anime,лол смотрю сейчас первую историю история про...,"[лол, смотрю, сейчас, первую, историю, история...","[лола, смотреть, сейчас, первый, история, исто...",лола смотреть сейчас первый история история пр...
245261,"Раньше была нормальная сметана, щас жидкая как...",2,perekrestok,раньше была нормальная сметана щас жидкая как...,"[раньше, была, нормальная, сметана, щас, жидка...","[ранний, быть, нормальный, сметана, сейчас, жи...",ранний быть нормальный сметана сейчас жидкий к...
116483,Горо - единственный нормальный человек в этой ...,2,anime,горо единственный нормальный человек в этой ...,"[горо, единственный, нормальный, человек, в, э...","[горо, единственный, нормальный, человек, в, э...",горо единственный нормальный человек в этот бо...
75710,"суки, товар не туда ушёл, деньги не вернули",2,rureviews,суки товар не туда ушёл деньги не вернули,"[суки, товар, не, туда, ушёл, деньги, не, верн...","[сука, товар, не, туда, уйти, деньга, не, верн...",сука товар не туда уйти деньга не вернуть


In [63]:
from sklearn.model_selection import train_test_split

In [64]:
y = df_small['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [65]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [66]:
y_pred = clf.predict(X_test)

In [67]:
from sklearn.metrics import accuracy_score

In [68]:
accuracy_score(y_test, y_pred)

0.6253333333333333

In [69]:
from sklearn.ensemble import RandomForestClassifier

In [70]:
model = RandomForestClassifier()

In [71]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [72]:
y_pred = model.predict(X_test)

In [73]:
accuracy_score(y_test, y_pred)

0.5706666666666667

In [76]:
from sklearn.naive_bayes import MultinomialNB

In [77]:
gnb = MultinomialNB()

# Train (fit) the model using the training data
gnb.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gnb.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)

In [78]:
accuracy

0.6026666666666667